# exp15

exp03 Tabnet entmax

https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import joblib
import random
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from itertools import combinations

import ipywidgets as widgets
import snappy


from pytorch_tabnet.pretraining import TabNetPretrainer

from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from pytorch_tabnet.metrics import Metric
# from ipywidgets import interact, Select


In [2]:

# # ====================================================
# # Get the difference
# # ====================================================
# def get_difference(data, num_features):
#     df1 = []
#     customer_ids = []
#     for customer_id, df in tqdm(data.groupby(['customer_ID'])):
#         # Get the differences
#         diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
#         # Append to lists
#         df1.append(diff_df1)
#         customer_ids.append(customer_id)
#     # Concatenate
#     df1 = np.concatenate(df1, axis = 0)
#     # Transform to dataframe
#     df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
#     # Add customer id
#     df1['customer_ID'] = customer_ids
#     return df1

# # ====================================================
# # Read & preprocess data and save it to disk
# # ====================================================
# def read_preprocess_data():
#     train = pd.read_parquet('/content/data/train.parquet')
#     features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
#     cat_features = [
#         "B_30",
#         "B_38",
#         "D_114",
#         "D_116",
#         "D_117",
#         "D_120",
#         "D_126",
#         "D_63",
#         "D_64",
#         "D_66",
#         "D_68",
#     ]
#     num_features = [col for col in features if col not in cat_features]
#     print('Starting training feature engineer...')
#     train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
#     train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
#     train_num_agg.reset_index(inplace = True)
#     train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
#     train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
#     train_cat_agg.reset_index(inplace = True)
#     train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
#     # Transform float64 columns to float32
#     cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
#     for col in tqdm(cols):
#         train_num_agg[col] = train_num_agg[col].astype(np.float32)
#     # Transform int64 columns to int32
#     cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
#     for col in tqdm(cols):
#         train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
#     # Get the difference
#     train_diff = get_difference(train, num_features)
#     train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
#     del train_num_agg, train_cat_agg, train_diff
#     gc.collect()
#     test = pd.read_parquet('../input/amex-fe/test_fe.parquet')
#     print('Starting test feature engineer...')
#     test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
#     test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
#     test_num_agg.reset_index(inplace = True)
#     test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
#     test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
#     test_cat_agg.reset_index(inplace = True)
#     # Transform float64 columns to float32
#     cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
#     for col in tqdm(cols):
#         test_num_agg[col] = test_num_agg[col].astype(np.float32)
#     # Transform int64 columns to int32
#     cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
#     for col in tqdm(cols):
#         test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
#     # Get the difference
#     test_diff = get_difference(test, num_featusres)
#     test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
#     del test_num_agg, test_cat_agg, test_diff
#     gc.collect()
#     # Save files to disk
#     train.to_parquet('../input/amex-fe/train_fe.parquet')
#     test.to_parquet('../input/amex-fe/test_fe.parquet')

# # Read & Preprocess Data
# read_preprocess_data()

# Training & Inference

In [3]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../feature/exp03_amex-fe/'
    output_dir = '../output/exp08 Tabnet lag feature/'
    seed = 46
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "tabnet"
    ver = "exp08"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
    return train, test

# ====================================================
# XGB train
# ====================================================

def cat_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf

def tabnet_train(x, y, xt, yt,cat_features):
    print("# of features:", x.shape[1])
    assert x.shape[1] == xt.shape[1]
    
    prams = {
        'depth': 8,
        'iterations':5000,
#         'learning_rate': 0.05,
        'random_state':CFG.seed,
    }

#     watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    watchlist = [(x, 'train'), (xt, 'eval')]
    clf = CatBoostClassifier(**prams)# 5000
    clf.fit(x, y, eval_set=[(xt, yt)], cat_features=cat_features,plot=True, verbose_eval = 100)
#     print('best ntree_limit:', clf.best_ntree_limit)
#     print('best score:', clf.best_score)
    # return clf.predict_proba(xt)[:, 1]
    return clf.predict_proba(xt)[:, 1], clf



# using amex metric to evaluate tabnet
class Amex_tabnet(Metric):
    
    def __init__(self):
        self._name = 'amex_tabnet'
        self._maximize = True

    def __call__(self, y_true, y_pred):
        amex = amex_metric_numpy(y_true, y_pred[:, 1])
        return max(amex, 0.)

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


def xgb_amex(y_pred, y_true):
    return 'amex', amex_metric_np(y_pred,y_true.get_label())


# Created by https://www.kaggle.com/yunchonggan
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def amex_metric_np(preds: np.ndarray, target: np.ndarray) -> float:
    indices = np.argsort(preds)[::-1]
    preds, target = preds[indices], target[indices]

    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_mask = cum_norm_weight <= 0.04
    d = np.sum(target[four_pct_mask]) / np.sum(target)

    weighted_target = target * weight
    lorentz = (weighted_target / weighted_target.sum()).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    n_pos = np.sum(target)
    n_neg = target.shape[0] - n_pos
    gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)

    g = gini / gini_max
    return 0.5 * (g + d)

def amex_metric_numpy(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

In [4]:
seed_everything(CFG.seed)
train, test = read_data()


In [5]:



# # ====================================================
# # Train & Evaluate
# # ====================================================


# cat_features = [
#     "B_30",
#     "B_38",
#     "D_114",
#     "D_116",
#     "D_117",
#     "D_120",
#     "D_126",
#     "D_63",
#     "D_64",
#     "D_66",
#     "D_68"
# ]
# cat_features = [f"{cf}_last" for cf in cat_features]
# for cat_col in cat_features:
#     encoder = LabelEncoder()
#     train[cat_col] = encoder.fit_transform(train[cat_col])
#     test[cat_col] = encoder.transform(test[cat_col])
# # Round last float features to 2 decimal place
# num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
# num_cols = [col for col in num_cols if 'last' in col]
# for col in num_cols:
#     train[col + '_round2'] = train[col].round(2)
#     test[col + '_round2'] = test[col].round(2)
# # Get the difference between last and mean
# num_cols = [col for col in train.columns if 'last' in col]
# num_cols = [col[:-5] for col in num_cols if 'round' not in col]
# for col in num_cols:
#     try:
#         train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
#         test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
#     except:
#         pass
# # Transform float64 and float32 to float16
# num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
# for col in tqdm(num_cols):
#     train[col] = train[col].astype(np.float16)
#     test[col] = test[col].astype(np.float16)


# #     params = {
# #         'objective': 'binary:logistic', 
# #         'tree_method': 'hist', #gpu_hist #hist
# #         'max_depth': 7,
# #         'subsample':0.88,
# #         'colsample_bytree': 0.5,
# #         'gamma':1.5,
# #         'min_child_weight':8,
# #         'lambda':70,
# #         'eta':0.03}


In [6]:
# Get feature list
features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
# all_data = features.extend("customer_ID")


train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

## infを含むデータを外れ値（－１００００）に置換
train = train.replace([np.inf, -np.inf],0)
test = test.replace([np.inf, -np.inf],0)

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

# cid = []
# cids = []

preds = []
cids = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    
#     cid = train["customer_ID"].loc[val_ind]
    cid = train["customer_ID"].loc[val_ind]
    cids.extend(cid)
    # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    # lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)

    
    model = TabNetClassifier(n_d = 32,
                         n_a = 32,
                         n_steps = 3,
                         gamma = 1.3,
                         n_independent = 2,
                         n_shared = 2,
                         momentum = 0.02,
                         clip_value = None,
                         lambda_sparse = 1e-3,
                         optimizer_fn = torch.optim.Adam,
                         optimizer_params = dict(lr = 1e-3, weight_decay=1e-3),
                         scheduler_fn = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
                         scheduler_params = {'T_0':5,
                                             'eta_min':1e-4,
                                             'T_mult':1,
                                             'last_epoch':-1},
                         mask_type = 'sparsemax',
                         seed = CFG.seed)

    print(f'start Training fold {fold}')
    model.fit(np.array(x_train),
                    np.array(y_train.values.ravel()),
                    eval_set = [(np.array(x_val), np.array(y_val.values.ravel()))],
                    max_epochs = 1,#50
                    patience = 1,#50
                    batch_size = 512,
                    eval_metric = ['auc', 'accuracy', Amex_tabnet]) # Last metric is used for early stoppin
    
    print(f'start Training fold {fold}')
    file_path = f"{CFG.output_dir}{CFG.model}_fold{fold}"
    saved_filepath = model.save_model(file_path)
    
#     oof_predictions[valid_idx] = model.predict_proba(x_val.values)[:, 1]
    preds.extend(model.predict_proba(np.array(x_val[features]))[:, 1])
    
#     preds.extend(oof_predictions)
    print("preds : ",len(preds))

    # Add to out of folds array
#     oof_predictions[val_ind] = val_pred
    
    # OOF用のcustomer_IDをExtend
    cids.extend(train["customer_ID"].loc[val_ind])
    
    # test predct
    pred = model.predict_proba(np.array(test[features]))[:, 1]
    preds.append(pred)
#     print(f'OOF score across folds: {amex_metric_numpy(target, oof_predictions.flatten())}')

#     # Compute fold metric
    del x_train, x_val, y_train, y_val
    gc.collect()


# # Compute out of folds metric
# score = amex_metric(train[CFG.target], preds)
# print(f'Our out of folds CV score is {score}')
# test_predictions = np.mean(preds,axis = 0)

# dic_oof = {
#     "customer_ID":cids,
#     "target":train[CFG.target],
#     "tabnet_oot":oof_predictions
# }

# # Create a dataframe to store out of folds predictions
# oof_df = pd.DataFrame(dic_oof)
# # oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
# oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store test prediction
# test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)



 
--------------------------------------------------
Training fold 0 with 1460 features...
start Training fold 0
epoch 0  | loss: 0.57556 | val_0_auc: 0.88237 | val_0_accuracy: 0.81341 | val_0_amex_tabnet: 0.54529 |  0:04:39s
epoch 1  | loss: 0.40267 | val_0_auc: 0.9087  | val_0_accuracy: 0.84091 | val_0_amex_tabnet: 0.60425 |  0:09:19s
epoch 2  | loss: 0.35697 | val_0_auc: 0.91864 | val_0_accuracy: 0.85069 | val_0_amex_tabnet: 0.62819 |  0:13:58s
epoch 3  | loss: 0.33654 | val_0_auc: 0.92087 | val_0_accuracy: 0.85545 | val_0_amex_tabnet: 0.63975 |  0:18:37s
epoch 4  | loss: 0.32599 | val_0_auc: 0.92358 | val_0_accuracy: 0.85753 | val_0_amex_tabnet: 0.64629 |  0:23:16s
epoch 5  | loss: 0.3123  | val_0_auc: 0.92962 | val_0_accuracy: 0.86406 | val_0_amex_tabnet: 0.66731 |  0:27:55s
epoch 6  | loss: 0.29858 | val_0_auc: 0.93434 | val_0_accuracy: 0.86867 | val_0_amex_tabnet: 0.68453 |  0:32:34s
epoch 7  | loss: 0.2869  | val_0_auc: 0.93894 | val_0_accuracy: 0.87417 | val_0_amex_tabnet: 0.7

In [7]:

# Compute out of folds metric
test_predictions = np.mean(test_preds,axis = 0)

# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


score = amex_metric(train[CFG.target], preds)
print(f'Our out of folds CV score is {score}')

dic_oof = {
    "customer_ID":cids,
    "target":train[CFG.target],
    "tabnet_oot":oof_predictions
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)



Our out of folds CV score is 0.020569053477342766


In [8]:
# # oof_predictions[valid_idx] = model.predict_proba(x_val.values)[:, 1]
# preds.extend(model.predict_proba(x_val.values)[:, 1])
# #     print("cid : ",len(cid))
# print("preds : ",len(preds))

# # Add to out of folds array
# # oof_predictions[valid_idx] = val_pred

# # OOF用のcustomer_IDをExtend
# cids.extend(train["customer_ID"].loc[val_ind])

# # test predct
# test_predictions += model.predict_proba(test.values)[:, 1]/5
# print(f'OOF score across folds: {amex_metric_numpy(target, oof_predictions.flatten())}')
